# Проект: прогнозирование уровня удовлетворённости и оттока сотрудников из компании. 

## Описание проекта

 HR-аналитики компании «Работа с заботой» помогают бизнесу оптимизировать управление персоналом: бизнес предоставляет данные, а аналитики предлагают, как избежать финансовых потерь и оттока сотрудников.

 Компания предоставила данные с характеристиками сотрудников компании. Среди них — уровень удовлетворённости сотрудника работой в компании. Эту информацию получили из форм обратной связи: сотрудники заполняют тест-опросник, и по его результатам рассчитывается доля их удовлетворённости от 0 до 1, где 0 — совершенно неудовлетворён, 1 — полностью удовлетворён. 
Собирать данные такими опросниками не так легко: компания большая, и всех сотрудников надо сначала оповестить об опросе, а затем проследить, что все его прошли. 

**Цель**

При помощи МО:

1. Построить модель, которая сможет предсказать уровень удовлетворённости сотрудника на основе данных заказчика. 
2. Построить модель, которая сможет на основе данных заказчика предсказать то, что сотрудник уволится из компании.

## Задача 1: предсказание уровня удовлетворённости сотрудника

**Описание данных**

- id — уникальный идентификатор сотрудника;
- dept — отдел, в котором работает сотрудник;
- level — уровень занимаемой должности;
- workload — уровень загруженности сотрудника;
- employment_years — длительность работы в компании (в годах);
- last_year_promo — показывает, было ли повышение за последний год;
- last_year_violations — показывает, нарушал ли сотрудник трудовой договор за последний год;
- supervisor_evaluation — оценка качества работы сотрудника, которую дал руководитель;
- salary — ежемесячная зарплата сотрудника;
- job_satisfaction_rate — уровень удовлетворённости сотрудника работой в компании, целевой признак.

Тренировочная выборка: train_job_satisfaction_rate.csv

Входные признаки тестовой выборки: test_features.csv

Целевой признак тестовой выборки: test_target_job_satisfaction_rate.csv

## Задача 2: предсказание увольнения сотрудника из компании

**Описание данных** 

Для этой задачи можно использовать те же входные признаки, что и в предыдущей задаче. 

Тренировочная выборка: train_quit.csv

Входные признаки тестовой выборки те же, что и в прошлой задаче: test_features.csv

Целевой признак тестовой выборки: test_target_quit.csv

**Этапы выволнения задач**

1. Загрузка и обзор данных.

2. Предобработка данных:
 - изучение данных - при необходимости выполнить предобработку. Если есть пропуски, заполним их в пайплайне.

3. Исследовательский анализ данных:
 - исследование все признаков.
 
4. Подготовка данных.

5. Обучение моделей.

6. Выводы.


# Задача 1

## 1. Загрузка данных

In [1]:
!pip install phik
!pip install shap
!pip install -U scikit-learn
!pip install "seaborn==0.13.2"

     |████████████████████████████████| 686 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 539 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.5 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 68.7 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
     |████████████████████████████████| 294 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 79.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 79.6 MB/s eta 0:00:01
     |████████████████████████████████| 321 kB 61.2 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing i

In [2]:
import pandas as pd
import seaborn as sns
import phik
import numpy as np
import scipy.stats as st
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
import shap


from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 

from phik import phik_matrix
from phik.report import plot_correlation_matrix

RANDOM_STATE = 100
TEST_SIZE = 0.25

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
train_job_satisfaction_rate = pd.read_csv('/datasets/train_job_satisfaction_rate.csv')
test_features = pd.read_csv('/datasets/test_features.csv')
test_job_satisfaction_rate = pd.read_csv('/datasets/test_target_job_satisfaction_rate.csv')

In [ ]:
def check_df(df):    
    display(df.info())   
    display(df.head())

In [ ]:
check_df(train_job_satisfaction_rate)
check_df(test_features)
check_df(test_job_satisfaction_rate)

**Вывод** на первый взгляд есть пропуски в датасете train_job_satisfaction_rate и test_features. Тип данных во всех датасетах соответствует данным в столбцах. Все названия столбцов находятся в едином регистре, исправлений не требуется.

## 2. Предобработка данных

Проверим количество пропусков и явных дубликатов.

In [ ]:
def check_df1(df):
    display(df.isna().sum())   
    display(df.duplicated().sum())

In [ ]:
check_df1(train_job_satisfaction_rate)
check_df1(test_features)
check_df1(test_job_satisfaction_rate)

Выведем строки с пропусками.

In [ ]:
train_job_satisfaction_rate[train_job_satisfaction_rate.isna().any(axis=1)]

In [ ]:
test_features[test_features.isna().any(axis=1)]

Заполним данные пропуски позже при построении паплайна. Явных дубликатов не обнаружено. Проверим неявные дубликаты.

In [ ]:
for column in train_job_satisfaction_rate.iloc[:, 1:8]:
    print(train_job_satisfaction_rate[column].unique())

In [ ]:
for column in test_features.iloc[:, 1:8]:
    print(test_features[column].unique())

Обнаружили пустые значения в датасете test_features. Выведен строки с этими значениями.

In [ ]:
test_features.query('dept == " "')

In [ ]:
test_features.query('workload == " "')

Так как данные пустые значения не относятся к значению Nan, но при этом не несут никакой информации, заполним их на Nan.

In [ ]:
test_features['dept'] = test_features['dept'].replace(' ', np.nan)
test_features['workload'] = test_features['workload'].replace(' ', np.nan)
test_features[test_features.isna().any(axis=1)]

Как видим, добавилось еще два пропуска. 

**Вывод**: в датасетах есть несколько пропусков. Явных и неявных дубликатов не обнаружено. Выявленные пустые значения заменили на значение Nan.

## 3. Исследовательский анализ данных

Используем функции для построения диаграмм для количественных и качественных признаков.

In [ ]:
def hist_box_plot(df, column):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(df[column], color='Green', kde=True, bins=25)
    plt.title(f'Гистограмма {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Количество')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[column], color='Red')
    plt.title(f'Диаграмма рассеяния {column}')
    plt.ylabel('Количество')
    
def bar_plot(df, column):
    plt.figure(figsize=(10,4))
    sns.catplot(data=df, y=column, hue=column, kind="count", palette="pastel")
    plt.title(f'Рапределение по {column}')
    plt.xlabel('Количество')
    plt.show()
   
def count_box_plot(df, column):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    sns.countplot(x=column, data=df)
    plt.title(f'Диаграмма {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Количество')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[column], color='Red')
    plt.title(f'Диаграмма рассеяния {column}')
    plt.ylabel('Количество')

Разделим количественные и качественные признаки для построения диаграмм. Учитываем, что имеющиеся количественные данные нужно разделить на дискретные и непрерывные.

In [ ]:
column_сat = ['dept',
              'level',
              'workload',
              'last_year_promo',
              'last_year_violations']

In [ ]:
column_num = ['salary',
              'job_satisfaction_rate']

Количественный признак для датасета test_features.

In [ ]:
column_num_salary = ['salary']

Количественный признак для датасета test_job_satisfaction_rate.

In [ ]:
column_num_rate= ['job_satisfaction_rate']

Дисретные количественные признаки.

In [ ]:
column_num_disc = ['employment_years',
                   'supervisor_evaluation']

### 3.1 Анализ датасета train_job_satisfaction_rate

Рассмотрим категориальные признаки графически и в цифрах.

In [ ]:
for column in column_сat:
    bar_plot(train_job_satisfaction_rate, column)

In [ ]:
for column in column_сat:
    print(train_job_satisfaction_rate[column].value_counts())

 - dept(отделы) - больше всего сотрудников в отделе продаж(sales) - 1512, в технологическом отделе(technology) 866 сотрудников, в отделе закупок(purchasing) - 610, отдел маркетинга(marketing ) -550, в HR отделе 456.
 - level(уровень должности) - больше всего в компании младших(junior) сотрудников - 1894, средних(middle) - 1744, меньше всего старших (sinior) сотрудников - 734.
 - workload - большая часть сотрудников в количестве 2066 имеют среднюю степень загрузки, 1200 сотрудников имеют низкую загрузку и 734 сотрудника высокую.
 - last_year_promo - за последний год повысили только 120 сотрудников, остальные 3880 остались на прежнем уровне.
 - last_year_violations - за последний год трудовой договор нарушило 559 сотрудников, 3441 работали без нарушений.

Проверим распределение количественных признаков.

In [ ]:
for column in column_num:
    hist_box_plot(train_job_satisfaction_rate, column)

- salary - заработная плата большинства сотрудников находится в диапазоне от 12 до 45 тысяч. Есть выбивающиеся значения выше 70 тысяч - вероятно, это заработная плата старших сотрудников.
- job_satisfaction_rate - удовлетворенность большинства сотрудников находится в диапазоне от 0.3 до 0.8. Достаточно сильный разброс в оценках.

In [ ]:
for column in column_num_disc:
    count_box_plot(train_job_satisfaction_rate, column)

In [ ]:
for column in column_num_disc:
    print(train_job_satisfaction_rate[column].value_counts())

- employment_years - большая часть сотрудников работает 1-2 года, 3-7 лет работают сотрудники в количестве от 483 до 287(по убыванию), в диапазоне 8-10 лет сотрудников от 193 до 91(по убыванию)
- supervisor_evaluation - по оценки руководителя большая часть сотрудников работает на 3 и 4 баллов из 5. 546 сотрудников получили оценку 5. Оценки 1 и 2 получили 201 и 387 сотрудников соответсвенно.

### 3.2 Корреляционный анализ датасета train_job_satisfaction_rate

Построим матрицу корреляции для оценки зависимости признаков. Для начала сделаем столбец id индексом.

In [ ]:
df_train = train_job_satisfaction_rate.set_index('id')
df_train.head()

In [ ]:
interval_cols = column_num

phik_overview = df_train.phik_matrix(interval_cols=interval_cols)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(10, 10)
) 

Мы видим относительно высокие коэффициенты корреляции между следущими признаками:
- job_satisfaction_rate и supervisor_evaluation - 0,76 - здесь не совсем очевидная корреляция. Чем луше начальник оценивает сотрудника, тем выше удовлетворенность. Может это связано с тем, что если сотрудник хорошо работает, начальство ценит, повышает заработную плату, возможно дают интересные проекты, соответственно и сотрудник удовлетворен работой.
- salary и workload - 0,79 - как правило здесь все логично для честный компаний - если сотрудник загружен, то и оплата труда должна быть выше
- salary и level - 0,72 - здесь такая же ситуация, логично , что старшие сотрудники имеют заработную плату выше , чем младшие
- employment_years и level - 0,68 - коэффициент не такой высокий, но все же - чем больше лет сотрудник работает, тем выше его уровень.

### 3.3 Анализ датасета test_features( входные признаки тестовой выборки)

In [ ]:
for column in column_сat:
    bar_plot(test_features, column)

In [ ]:
for column in column_сat:
    print(test_features[column].value_counts())

 - dept(отделы) - больше всего сотрудников в отделе продаж(sales) - 763, в технологическом отделе(technology) 455 сотрудников, в отделе закупок(purchasing) - 273, отдел маркетинга(marketing ) -279, в HR отделе 227.
 - level(уровень должности) - больше всего младших(junior) сотрудников - 974, средних(middle) - 854, меньше всего старших (sinior) сотрудников - 171.
 - workload - большая часть сотрудников в количестве 1043 имеют среднюю степень загрузки, 593 сотрудника имеют низкую загрузку и 363 сотрудника высокую.
 - last_year_promo - за последний год повысили 63 сотрудникf, остальные 1937 остались на прежнем уровне.
 - last_year_violations - за последний год трудовой договор нарушило 262 сотрудников, 1738 работали без нарушений.

Проверим распределение количественных признаков.

In [ ]:
for column in column_num_salary:
    hist_box_plot(test_features, column)

In [ ]:
for column in column_num_disc:
    count_box_plot(test_features, column)

In [ ]:
for column in column_num_disc:
    print(test_features[column].value_counts())

- salary - заработная плата большинства сотрудников находится в диапазоне от 12 до 40 тысяч. Такие же выбивающиеся значения выше 70 тысяч - вероятно, это заработная плата старших сотрудников.
- employment_years - большая часть сотрудников работает от 1 до 6 лет, преобладающая часть из них 1-2 года.
- supervisor_evaluation - по оценки руководителя большая часть сотрудников работает на 3 и 4 баллов из 5. 284 сотрудника получили оценку 5. Оценки 1 и 2 получили 88 и 196 сотрудников соответсвенно.

### 3.4 Анализ датасета test_job_satisfaction_rate(целевой признак тестовой выборки)

In [ ]:
for column in column_num_rate:
    hist_box_plot(test_job_satisfaction_rate, column)

- job_satisfaction_rate - удовлетворенность большинства сотрудников находится в диапазоне от 0.4 до 0.7. Если смотреть общую картину, то наблюдается достаточно сильный разброс в оценках сотрудников.

Теперь для удобства объединим датасеты тестовой выборки - test_features и test_job_satisfaction_rate. Заменим id на индекс.

In [ ]:
df_test = test_features.merge(test_job_satisfaction_rate, on='id')
df_test = df_test.set_index('id')
df_test.head()

In [ ]:
df_test.info()

Объединение прошло успешно.

### 3.5 Корреляционный анализ датасета train_job_satisfaction_rate

Теперь проведем корреляционный анализ тестовой выборки.

In [ ]:
interval_cols = column_num

phik_overview = df_test.phik_matrix(interval_cols=interval_cols)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(10, 10)
) 

Похожие зависимости как и у тренировочной выборки, а именно: 
- job_satisfaction_rate и supervisor_evaluation - 0,77 
- salary и level - 0,75
- salary и workload - 0,79
- employment_years и level - 0,69

**Вывод:** был проведен исследовательский анализ всех датасетов. Картина такова : в компании много молодых сотрудников на позиции junior - это может говорить о том, что в компании большая текучка кадров, молодые специалисты не задерживаются в компании. Опытных специалистов sinior достаточно мало. Повышения в компании случаются редко, несмотряна то, что много сотрудников по оценке руководителя работает на 4. Также большинство сотрудников работают без нарушения трудового договора. Корреляционный анализ показал - чем больше нагрузка, опыт, тем выше заработная плата, чем больше стаж , тем выше уровень.Более подробные выводы  находятся в подпунктах данного пункта.


## 4. Подготовка данных

Обозначим тренировные и тестовые выборки. 

In [ ]:
X_train = df_train.drop('job_satisfaction_rate', axis=1)
y_train = df_train['job_satisfaction_rate']

X_test = df_test.drop('job_satisfaction_rate', axis=1)
y_test = df_test['job_satisfaction_rate']

Введем обозначения для типов исходных данных.

In [ ]:
ohe_columns = ['dept',
               'last_year_violations',
               'last_year_promo']

ord_columns = ['level',
               'workload']

num_columns = ['salary',
               'employment_years',
               'supervisor_evaluation']

Создаем пайплайны для подготовки признаков.

In [ ]:
ohe_pipe = Pipeline(
    [
        ('simpleImputer_ohe',
         SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        ('ohe',
         OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
        )
    ]
 )

In [ ]:
ord_pipe = Pipeline(
    [
        (
            'simple_imputer_ord_before',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(categories=[
                                      ['junior', 'middle', 'sinior'],
                                      ['low', 'medium', 'high']],
                          handle_unknown='use_encoded_value',
                          unknown_value=np.nan)
        ),
        (
            'simple_imputer_ord_after',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

Создаем общий пайплайн для подготовки данных.

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ],
        remainder='passthrough'
)

Создаем итоговый пайплайн.

In [ ]:
pipe_final = Pipeline(
    [
        ('preprocessor', 
         data_preprocessor
        ),
        ('models',
        LinearRegression(n_jobs=-1)
        )
    ]
)

Создаем словарь гиперпараметров с моделями DecisionTreeRegressor(), LinearRegression().

In [ ]:
param_grid = [
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 15),
        'models__max_features': range(2, 15),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
 
    {
        'models': [LinearRegression(n_jobs=-1)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]

## 5. Обучение моделей

Выберем лучшую модель, используя метрику — SMAPE (англ. symmetric mean absolute percentage error, «симметричное среднее абсолютное процентное отклонение»). 

Метрика SMAPE вычисляется так:

$$
SMAPE = \frac{100}{n} \sum_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{(|y_i| + |\hat{y}_i|)/2} \
$$

где:

- $y_i$ - фактическое значение целевого признака для объекта с порядковым номером $i$ в выборке;
- $\hat{y}_i$ - предсказанное значение целевого признака для объекта с порядковым номером $i$ в выборке;
- $n$ - количество объектов в выборке;
- $\sum_{i=1}^{n}$ - сумма значений, полученная в результате операций, которые следуют за этим знаком, для всех объектов с порядковым номером от $i$ до $n$ в выборке.

Напишем функцию, которая возвращает значение метрики SMAPE. 

In [ ]:
def smape(y_true, y_pred):
    n = len(y_true)
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    summ = np.sum(numerator / denominator)
    smape_value = summ * (100 / n)
    return smape_value

Создадим scorer для метрики SMAPE

In [ ]:
smape_scorer = make_scorer(score_func=smape, greater_is_better=False)

In [ ]:
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid,
    cv=5,
    scoring=smape_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

In [ ]:
randomized_search.fit(X_train, y_train)

In [ ]:
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print (f'Метрика лучшей модели по кросс-валидации на обучающих данных {round(randomized_search.best_score_*(-1),4)}')

In [ ]:
y_test_pred = randomized_search.predict(X_test)
print(f'Метрика SMAPE на тестовой выборке: {round(smape(y_test, y_test_pred), 4)}')

Согласно заданию критерий успеха: SMAPE ≤ 15 на тестовой выборке. В нашем случае лучшая модель DecisionTreeRegressor показала метрику SMAPE= 14.3134, что говорит нам об успехе.

## 6. Промежуточный вывод

Для поиска лучшей модели было сделано следующее:

- проведен исследовательский анализ всех датасетов
- подготовили данные :для обучения использовали 2 модели DecisionTreeRegressor и LinearRegression. Использовали 2 кодировщика.
- лучшая модель оказалась DecisionTreeRegressor(max_depth=12, max_features=12, random_state=100)
- метрика SMAPE на обучающих данных составила 15.2937, на тестовых 14.3134.

# Задача 2

Перейдем к построению модели, которая сможет на основе данных заказчика предсказать то, что сотрудник уволится из компании.
Для этой задачи будем использовать те же входные признаки, что и в предыдущей задаче. Однако целевой признак отличается: это quit — увольнение сотрудника из компании.

## 1. Загрузка данных

Так как входные признаки тестовой выборки те же, что и в прошлой задаче 
test_feature, загрузим только датасет train_quit(тренировочная выборка) и test_target_quit(целевой признак тестовой выборки).
v

In [ ]:
train_quit = pd.read_csv('/datasets/train_quit.csv')
test_target_quit = pd.read_csv('/datasets/test_target_quit.csv')

In [ ]:
check_df(train_quit)
check_df(test_target_quit)

**Вывод** тип данных во всех датасетах соответствует данным в столбцах. Все названия столбцов находятся в едином регистре, исправлений не требуется. Пропусков нет.

## 2. Предобработка данных

Проверим количество пропусков и явных дубликатов только в двух датасетах train_quit и test_target_quit. 

In [ ]:
check_df1(train_quit)
check_df1(test_target_quit)

Пропусков и явных дубликатов нет. Проверим неявные дубликаты.

In [ ]:
for column in train_quit.iloc[:, 1:10]:
    print(train_quit[column].unique())

In [ ]:
for column in test_target_quit.iloc[:, 1:2]:
    print(test_target_quit[column].unique())

**Вывод**: в датасетах нет пропусков. Явных и неявных дубликатов не обнаружено.

## 3. Исследовательский анализ данных

Распределим данные на категориальные и количественные признаки с учетом quit. Дискретные количествееные признаки будем использовать те же - column_num_disc.

In [ ]:
column_quit_сat = ['dept',
                   'level',
                   'workload',
                   'last_year_promo',
                   'last_year_violations',
                   'quit']

In [ ]:
column_quit_num = ['salary']

In [ ]:
column_quit_test_cat = ['quit']

### 3.1 Анализ датасета train_quit

Рассмотрим категориальные признаки.

In [ ]:
for column in column_quit_сat:
    bar_plot(train_quit, column)

In [ ]:
for column in column_quit_сat:
    print(train_quit[column].value_counts())

 - dept(отделы) - больше всего сотрудников в отделе продаж(sales) - 1438, в технологическом отделе(technology) 928 сотрудников, в отделе закупок(purchasing) - 588, отдел маркетинга(marketing ) -582, в HR отделе 464.
 - level(уровень должности) - больше всего в компании младших(junior) сотрудников - 1949, средних(middle) - 1694, меньше всего старших (sinior) сотрудников - 357.
 - workload - большая часть сотрудников в количестве 2118 имеют среднюю степень загрузки, 1208 сотрудников имеют низкую загрузку и 674 сотрудника высокую.
 - last_year_promo - за последний год повысили только 113 сотрудников, остальные 3887 остались на прежнем уровне.
 - last_year_violations - за последний год трудовой договор нарушило 545 сотрудников, 3455 работали без нарушений.
 - quit - уволилось 1128 сотрудников, 2872 нет

Проверим распределение количественных признаков.

In [ ]:
for column in column_quit_num:
    hist_box_plot(train_quit, column)

Данные аналогичны анализу датасета train_job_satisfaction_rate:
- salary - заработная плата большинства сотрудников находится в диапазоне от 12 до 45 тысяч. Есть выбивающиеся значения выше 75 тысяч - вероятно, это заработная плата старших сотрудников.

In [ ]:
for column in column_num_disc:
    count_box_plot(train_quit, column)

Данные аналогичны анализу датасета train_job_satisfaction_rate:
- employment_years - большая часть сотрудников работает 1-2 года, дальше по убыванию
- supervisor_evaluation - по оценки руководителя большая часть сотрудников работает на 3 и 4 баллов из 5.

### 3.2 Корреляционный анализ датасета train_quit

Проведем корреляционный анализ. Для начала столбец id сделаем индексом.

In [ ]:
df_train_quit = train_quit.set_index('id')
df_train_quit.head()

In [ ]:
interval_cols = column_num

phik_overview = df_train_quit.phik_matrix(interval_cols=interval_cols)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(10, 10)
) 

Мы видим относительно высокие коэффициенты корреляции между следущими признаками:
- salary и workload - 0,79 - похожий коэффициент как в задаче 1, чем выше загруженность , тем  выше зарплата.
- salary и level - 0,75 - такая же ситуация, чем выше уровень , тем  выше зарплата.
- employment_years и level - 0,69 - коэффициент не такой высокий, но все же - чем больше лет сотрудник работает, тем выше его уровень
- quit и employment_years - 0,66 - коэффициент еще ниже, но все же зависимость есть - чем больше лет сотрудник работает, тем больше вероятность, что он уволится.

### 3.3 Составление портрета «уволившегося сотрудника» датасета train_quit

Проверим, в каком отделе с большей вероятностью работает уволившийся сотрудник, его уровень загруженности, уровень должности, было ли повышение и нарушения трудового договора.

In [ ]:
plt.figure(figsize=(15, 15))

for a, column in enumerate(column_сat, start=1):
    plt.subplot(3, 2, a)
    train_quit_group = df_train_quit.groupby(column)['quit'].value_counts(normalize=True).rename('proportion').reset_index()
    train_quit_group['proportion'] *= 100
    ax = sns.barplot(x=column, y='proportion', hue='quit', data=train_quit_group, palette='husl')
    ax.set_ylabel('Процент сотрудников')
    ax.yaxis.set_major_formatter(plt.FuncFormatter('{:.0f}%'.format))
    plt.suptitle('Соотношение уволившихся и работающих сотрудников', fontsize=20)


plt.show()

- dept - количество уволившихся сотрудников примерно одинаковое по всем отделам, чуть меньше в HR отделе
- level - больше всего увольняются младшие сотрудники, уволившихся middle и sinior мало
- workload - у большинства уволившихся сотрудников нагрузка маленькая
- last_year_promo - около 30% уволившихся сотрудников не получали повышение
- last_year_violations - среди уволившихся есть  те, кто нарушал трудовой договор, и те, кто не нарушал. Конечно больше тех, кто нарушал.

Сравним среднее значение зарплаты ушедших сотрудников с теми, кто остался в компании.

In [ ]:
plt.figure(figsize=(15, 15))

for a, column in enumerate(column_сat, start=1):
    plt.subplot(3, 2, a)
    sns.barplot(x=column, y='salary', hue='quit', data=df_train_quit, palette='bwr')
    ax.set_ylabel('Зарплата')
    plt.suptitle('Соотношение зарплаты уволившихся и работающих сотрудников', fontsize=20)


plt.show()

- dept - у уволившихся сотрудников по всем отделам зарплата была ниже, чем у оставшихся
- 
level -у уволившихся сотрудников по всем уровням зарплата была ниже, чем у оставшихсяо- 
workload при одинаковой нагрузке ува уволившихсзарплата нижеа- я
last_year_promoу уволившихся зарплата нижен- ие
last_year_violationу уволившихся зарплата ниже

Теперь перейдем к анализу тестовой выборки с целевым признаком quit. Так как входные признаки тестовой выборки были проанализированы в задаче 1 test_features, мы визуализируем и сравненим распределения признака job_satisfaction_rate.шал.

### 3.4 Визуализация и сравнение распределения признака job_satisfaction_rate для ушедших и оставшихся сотрудников. 

Аналитики утверждают, что уровень удовлетворённости сотрудника работой в компании влияет на то, уволится ли сотрудник. Проверим это. Используем данные с обоими целевыми признаками тестовой выборки. Для начала объединим датасеты test_features и test_target_quit в датасет df_test_quit, он нам понадобится позже.

In [ ]:
df_test_quit = test_features.merge(test_target_quit, on='id').set_index('id')
df_test_quit.head()

In [ ]:
df_test_quit.info()

Теперь объединим датасет тестовой выборки с целевым признаком задачи 2 df_test_quit с датасетом целевого признака задачи 1 test_job_satisfaction_rate.

In [ ]:
df_test_quit_rate = df_test_quit.merge(test_job_satisfaction_rate, on='id').set_index('id')
df_test_quit_rate.head()

In [ ]:
df_test_quit_rate.info()

In [ ]:
plt.figure(figsize=(10, 10))
sns.kdeplot(data=df_test_quit_rate, x="job_satisfaction_rate", hue="quit", multiple="stack", alpha=.5)
plt.title('Распределения признака "Уровень удовлетворенности сотрудника" для ушедших и оставшихся сотрудников')
plt.ylabel('Плотность распределения')
plt.xlabel('Уровень удовлетворенности сотрудников')
plt.show()

График показывает, что уволившиеся сотрудники в основном не были удовлетворены работой. У большинства оценка от 0,1 до 0,5. Сотрудники, которые остались, оценивают свою удовлетворенность от 0,5 и выше.

### 3.5 Корреляционный анализ датасета df_test_quit_rate

Теперь проведем корреляционный анализ тестовой выборк, разбив ее на тех, кто уволился и тех, кто осталсяи Данная мера может показать зависимости, которые вероятно могут повлиять на увольнение.


In [ ]:
df_test_quit_rate_yes = df_test_quit_rate.query("quit == 'yes'")
df_test_quit_rate_no = df_test_quit_rate.query("quit == 'no'")

In [ ]:
phik_overview_yes = df_test_quit_rate_yes.drop(columns=['quit']).phik_matrix(interval_cols=interval_cols)
phik_overview_no = df_test_quit_rate_no.drop(columns=['quit']).phik_matrix(interval_cols=interval_cols)

plot_correlation_matrix(
    phik_overview_yes.values,
    x_labels=phik_overview_yes.columns,
    y_labels=phik_overview_yes.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'Корреляция $\phi_K$ уволившегося сотрудника',
    fontsize_factor=1.5,
    figsize=(10, 10)
);

plot_correlation_matrix(
    phik_overview_no.values,
    x_labels=phik_overview_no.columns,
    y_labels=phik_overview_no.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'Корреляция $\phi_K$ оставшегося сотрудника',
    fontsize_factor=1.5,
    figsize=(10, 10)
);


Коэффициенты корреляции уволившихся сотрудников :

- salary и workload - 0,81 
- salary и level - 0,71 
- salary и employment_years - 0,75
- supervisor_evaluation и job_satisfaction_rate - 0,76
- last_year_promo и employment_years - 0,72
- employment_years и level - 0,73

Коэффициенты корреляции ставшихся сотрудников :

- salary и workload - 0,78 
- salary и level - 0,73
- supervisor_evaluation и job_satisfaction_rate - 0,77
- last_year_promo и employment_years - 0,72
- employment_years и level - 0,65

Все зависимости логичны: чем выше нагружка, уровень и стаж, чем выше зарплата; чем выше оценка руководства, тем выше удовлетворенность сотрудника; чем больше сотрудник работает, тем очевиднее повышение в зарплате и должности. В целом коэффициентв двух групп не сильно отличаются друг от друга.

## 4. Добавление нового входного признака

Допустим, job_satisfaction_rate и quit действительно связаны. Добавим job_satisfaction_rate, предсказанный лучшей моделью первой задачи, к входным признакам второй задачи.

In [ ]:
df_train_quit['job_satisfaction_rate_pred'] = randomized_search.best_estimator_.predict(df_train_quit)
df_train_quit.head()

In [ ]:
df_train_quit.info()

In [ ]:
df_test_quit['job_satisfaction_rate_pred'] = randomized_search.best_estimator_.predict(df_test_quit)
df_test_quit.head()

In [ ]:
df_test_quit.info()

## 5. Подготовка данных

Обозначим тренировные и тестовые выборки. 

In [ ]:
X_train = df_train_quit.drop('quit', axis=1)
y_train = df_train_quit['quit']

X_test = df_test_quit.drop('quit', axis=1)
y_test = df_test_quit['quit']

Введем обозначение количественных признаков с учетом добавленного job_satisfaction_rate_pred. Категориальные признаки ohe_columns и ord_columns остаются прежними как в задаче 1. 

In [ ]:
num_columns = ['salary',
               'employment_years',
               'supervisor_evaluation',
               'job_satisfaction_rate_pred']

Соберем паплайн.

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

ohe_pipe = Pipeline(
    [
        ('simpleImputer_ohe',
         SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        ('ohe',
         OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
        )
    ]
 )

ord_pipe = Pipeline(
    [
        (
            'simple_imputer_ord_before',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(categories=[
                                      ['junior', 'middle', 'sinior'],
                                      ['low', 'medium', 'high']],
                          handle_unknown='use_encoded_value',
                          unknown_value=np.nan)
        ),
        (
            'simple_imputer_ord_after',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ],
        remainder='passthrough'
)

pipe_final = Pipeline(
    [
        ('preprocessor', 
         data_preprocessor
        ),
        ('models',
        DecisionTreeClassifier(random_state=RANDOM_STATE)
        )
    ]
)

Создаем словарь гиперпараметров с моделями DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression().

In [ ]:
param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 15),
        'models__max_features': range(2, 15),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']   
    },

    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]

## 6. Обучение модели

Метрика оценки качества в этой задаче — ROC-AUC. Критерий успеха: ROC-AUC ≥0.91 на тестовой выборке.

In [ ]:
grid_search = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print (f'Метрика лучшей модели по кросс-валидации на обучающих данных {round(grid_search.best_score_,4)}')

In [ ]:
print(f'Метрика ROC-AUC лучшей модели на тестовой выборке: {round(roc_auc_score(y_test, grid_search.predict_proba(X_test)[:, 1]),4)}')

Критерий успеха достигнут, ROC-AUC лучшей модели на тестовой выборке: 0.9185.

## 7. Промежуточный вывод

Для поиска лучшей модели было сделано следующее:

- проведен исследовательский анализ всех датасетов
- подготовили данные :для обучения использовали 3 модели DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression(). Использовали 2 кодировщика.
- лучшая модель оказалась DecisionTreeClassifier(max_depth=5, max_features=7, random_state=100)
- метрика ROC-AUC на обучающих данных составила 0.9268, на тестовых 0.9185.

# Общий вывод

**Описание задачи:**

При помощи МО:

- Построить модель, которая сможет предсказать уровень удовлетворённости сотрудника на основе данных заказчика.
- Построить модель, которая сможет на основе данных заказчика предсказать то, что сотрудник уволится из компании.
 
**Описание этапов работы:**

- Предобработка и анализ данных: была проведена оценка пропусков, дубликатов, анализ всех данных, выбросов, зависимость корреляции между признаками.
- Подготовка данных: были обработаны пропущенные значения, провели кодирование признаков категориальных переменных, построили паплайн.
- Выбор модели: было выбрано несколько моделей машинного обучения. Для задачи 1 DecisionTreeRegressor и LinearRegression и для задачи 2 DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression(). 
- Для задачи 1: лучшая модель оказалась DecisionTreeRegressor(max_depth=12, max_features=12, random_state=100), метрика SMAPE на обучающих данных составила 15.2937, на тестовых 14.3134 - данная модель позволит компании спрогнозировать уровень удовлетворенности сотрудников.
- Для задачи 2: лучшая модель оказалась DecisionTreeClassifier(max_depth=5, max_features=7, random_state=100), метрика ROC-AUC на обучающих данных составила 0.9268, на тестовых 0.9185 - данная модель предскажеть вероятность увольнения сотрудника.

**Предложения для бизнеса:**

При помощи данных моделей компания может выявлять потенциально недовольных сотрудников и тех, кто вероятно может уволиться. Основываясь на прогнозах, можно создать мотивационные/бонусные программы. Например, можно внедрить соц.пакеты - дмс, оплата питания, фитнес центра. Так как увольняется много молодых специалистов, то для таких сотрудников важно развитие в карьере - интересные интерактивные воркшопы, тренинги, приглашение сторонних опытных спикеров, возможность посещения выездных конференций- все это будет мотивацией для сотрудников продолжать работать и развиваться в одной компании.